In [1]:
import re
import json
import requests
from getpass import getpass
from scipy import ndimage
from skimage.segmentation import clear_border
from skimage.color import label2rgb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import cv2

from omero_tools import OmeroConnect
from matolab_tools import annotate_csv_upload, annotate_csv_uri, csvw_to_rdf, get_joined_rdf, create_mapping
# need credentials
username = input("Username: ")
password = getpass("OMERO Password: ")

HOST = 'wss://wss.omero.matolab.org'
omero_conn=OmeroConnect(HOST,username,password)

def run_detection(
        image,
        threshold_method: str = "Otsu",
        size_thresh: float = 80,
        dilate_kernel_size: int = 3,
        median_filter_radius: int = 4,
        plot=False
):

    name = image.name
    print("Image Name: "+name)

    gray = omero_conn.get_grayscale(image.id)

    print("Grayscale image shape:", gray.shape)
    print("Grayscale image data type:", gray.dtype)

    # Apply median filter using OpenCV

    #selem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*median_filter_radius + 1, 2*median_filter_radius+1))
    filtered = cv2.medianBlur(gray, 2 * median_filter_radius + 1)

    # Display the original and filtered images side by side
    if plot:
        fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))
        ax0.imshow(gray, cmap='gray')
        ax0.set_title('Step1: Load Original Image')
        ax0.axis('off')
        ax1.imshow(filtered, cmap='gray')
        ax1.set_title('Step2: Median Blur Filter')
        ax1.axis('off')

    # Apply Otsu's thresholding using OpenCV
    if threshold_method == "Otsu":
        ret, thresh = cv2.threshold(
            filtered, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        print('Threshold value is {}'.format(ret))
    else:
        print("only Otsu thresholding supported, skippping thresholding")
        thresh = gray

    # Dilate the thresholded image using a 3x3 kernel
    kernel = np.ones((dilate_kernel_size, dilate_kernel_size), np.uint8)
    dilated = cv2.dilate(thresh, kernel)

    if plot:
        fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))
        ax0.imshow(thresh, cmap='gray')
        ax0.set_title('Step3: Otsu Threshold')
        ax0.axis('off')
        ax1.imshow(dilated, cmap='gray')
        ax1.set_title('Step4: Dilated Image')
        ax1.axis('off')

    # Remove small objects using OpenCV's morphologyEx function
    morphed = cv2.morphologyEx(dilated, cv2.MORPH_OPEN, kernel, iterations=1)
    contours, hierarchy = cv2.findContours(
        morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < size_thresh:
            cv2.drawContours(morphed, [cnt], 0, 0, -1)

    # delete all previously defined shapes
    omero_conn.delete_rois(image.id)

    # upload shapes to omero
    omero_conn.create_omero_roi_polygons(image.id, contours)

    # Apply clear border to the dilated image
    mask = morphed == 255
    mask = clear_border(mask)

    # Label the mask and count the number of precipitates detected
    s = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
    labeled_mask, num_labels = ndimage.label(mask, structure=s)
    img2 = label2rgb(labeled_mask, bg_label=0)

    # Display the labeled mask
    if plot:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(img2)
        ax.set_title('Step5: Contour Finding')
        ax.axis('off')
        plt.show()
        print(f"Number of precipitates detected: {num_labels}")

    plt.show()




Connected as thomas.hanke
User ID: 102
User Full Name: Thomas Hanke
Your Groups:
   Name: matolab  ID: 53
   Name: kupferdigital  ID: 54
Current group:  matolab


## The Pipeline

In [2]:
#load all images from selected datasets, run detection algorithm and upload found contours as polygons in a omero roi
#results are saved as detection_runs.csv
datasets=omero_conn.get_datasets(51)
data=list()
for dataset in datasets:
    print(dataset.name)
    images = omero_conn.get_images(dataset.id)
    for image in images:
        startTime = datetime.now().isoformat()
        info = image.name.split(".",1)[0].split('_')
        state=info[0].split("-")
        if len(state)>1:
            anneal_temp, anneal_time = info[0].split("-")
            anneal_temp = float(anneal_temp.rsplit('C',1)[0])
            anneal_time = float(anneal_time.rsplit('h',1)[0])
        else:
            anneal_temp, anneal_time = 23.0, 0.0
        specimen = info[1].split('Sample',1)[-1]
        pos = info[2].split('Sample',1)[-1]
        threshold_method="Otsu"
        size_thresh=80
        dilate_kernel_size=3
        median_filter_radius=4
        info_dict={
            "SpecimenName": dataset.name+'_'+specimen,
            "Aging Temp [°C]": float(anneal_temp),
            "Aging Time [h]": float(anneal_time),
            "Creep Stress [MPa]": 0,
            "Dataset": meta_extractor_api+"dataset/"+str(dataset.id),
            "Position_Id": pos,
            "Image": meta_extractor_api+"image/"+str(image.id),
            "ROIs": meta_extractor_api+"rois/"+str(image.id),
            "DiskRadiusValue [px]": median_filter_radius,
            "Threshold Method": threshold_method,
            "Dilation Kernel Size [px]": dilate_kernel_size,
            "Date": startTime
        }
        run_detection(image,threshold_method="Otsu",size_thresh=80,dilate_kernel_size=3,median_filter_radius=4)
        data.append(info_dict)
        #break
    #break
df=pd.DataFrame(data)
df.to_csv('detection_runs.csv')

190C_1000h
---- Loaded image ID: 83
---- Loaded image ID: 84
---- Loaded image ID: 85
---- Loaded image ID: 74
---- Loaded image ID: 75
---- Loaded image ID: 76
---- Loaded image ID: 77
---- Loaded image ID: 78
---- Loaded image ID: 79
---- Loaded image ID: 80
---- Loaded image ID: 81
---- Loaded image ID: 82
Dataset ID: 53
Dataset Name: 190C_1000h
Image Name: 190C-1000h_Sample1_Stelle 10 DF 30s.dm3
Grayscale image shape: (2048, 2048)
Grayscale image data type: uint8
Threshold value is 61.0
deleted rois: [542]
added 33 polygon shapes to image


# Processing Metadata

In [3]:
# annotate detection_runs.csv
response=annotate_csv_uri("https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs.csv")

csvw annotation file created, suggested name: detection_runs-metadata.json
wrote csvw meta data to detection_runs-metadata.json


In [8]:
# serialize table to rdf, uses already commited files on main branch
meta_url="https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs-metadata.json"
response=csvw_to_rdf(meta_url)

writen serialized table to detection_runs.ttl


In [5]:
# create a rule bases mapping between the data in detection_runs and the precipitate analysis knowledge graph 
meta_url="https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs-metadata.json"
method_url="https://github.com/BAMresearch/DF-TEM-PAW/raw/main/PrecipitateAnalysisWorkflow.ttl"
d_classes= [
    "http://www.w3.org/ns/oa#Annotation",''
    "http://www.w3.org/ns/csvw#Column"
]
m_classes=["https://w3id.org/pmd/co/ValueObject",]
pred="https://w3id.org/pmd/co/isResourceOf"
map_dict={
    "diskRadius": "table-1-DiskradiusvaluePx",
    "kernelSize": "table-1-DilationKernelSizePx",
    "thresholdMethod": "table-1-ThresholdMethod",
    "specimenAgingTemperature": "table-1-AgingTempC",
    "specimenCreepStress": "table-1-CreepStressMpa",
    "investigationPosition": "table-1-Position_Id",
    "specimenName": "table-1-Specimenname",
    "specimenAgingTime": "table-1-AgingTimeH",
    "darkfieldTransmissionElectronMicroscopeImage": "table-1-Image",
    "precipitateRegion": "table-1-Rois",
    "executionDate": "table-1-Date",
}
create_mapping(meta_url=meta_url,method_url=method_url,data_super_classes=d_classes,predicate=pred,method_super_classes=m_classes,map_dict=map_dict)

writen mapping file to detection_runs-map.yaml


True

In [9]:
# join all data and replicate template knowledge graph for every row in table
mapping_url = "https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs-map.yaml"
data_url = "https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs.ttl"
duplicate_for_table = True
get_joined_rdf(map_url=mapping_url,data_url=data_url,duplicate_for_table=duplicate_for_table)


applied 10 mapping rules and skipped 0
wrote joint graph to detection_runs-joined.ttl
